# Halimbawa ng Paggamit ng Semantic Kernel Tool

Ang dokumentong ito ay nagbibigay ng pangkalahatang-ideya at paliwanag tungkol sa code na ginamit upang lumikha ng isang tool na nakabatay sa Semantic Kernel na isinama sa Azure AI Search para sa Retrieval-Augmented Generation (RAG). Ipinapakita ng halimbawa kung paano bumuo ng isang AI agent na kumukuha ng mga dokumento sa paglalakbay mula sa isang Azure AI Search index, nagpapahusay ng mga tanong ng user gamit ang mga resulta ng semantic search, at nag-i-stream ng detalyadong mga rekomendasyon sa paglalakbay.


### Pag-import ng mga Package
Ang sumusunod na code ay nag-i-import ng mga kinakailangang package:


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### Paglikha ng Semantic Kernel at AI Service

Ang isang Semantic Kernel na instance ay nilikha at na-configure gamit ang isang asynchronous na OpenAI chat completion service. Ang serbisyo ay idinaragdag sa kernel para magamit sa pagbuo ng mga sagot.


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### Pagpapakilala sa Prompt Plugin

Ang PromptPlugin ay isang katutubong plugin na nagtatakda ng isang function upang bumuo ng isang pinalawak na prompt gamit ang retrieval context


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## Pagsisimula ng Vector Database

Inilulunsad natin ang Azure AI Search na may permanenteng imbakan at nagdadagdag ng mga pinahusay na sample na dokumento. Ang Azure AI Search ay gagamitin upang mag-imbak at maghanap ng mga dokumento na nagbibigay ng konteksto para sa pagbuo ng tamang mga sagot.


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### Pagpapatakbo ng Ahente gamit ang Streaming Invocation

Ang pangunahing asynchronous loop ay gumagawa ng thread para sa pag-uusap, at para sa bawat input ng user, upang makita ng ahente ang retrieval context. Ang mensahe ng user ay idinadagdag din, at pagkatapos ay tinatawag ang ahente gamit ang streaming. Ang output ay ipinapakita habang ito ay dumadaloy.


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


# Mga Pangunahing Konsepto

Ang dokumentong ito ay naglalaman ng mga pangunahing impormasyon na makakatulong sa iyong maunawaan ang mga pangunahing aspeto ng aming sistema.

## Layunin

Ang layunin ng dokumentong ito ay magbigay ng malinaw at maikling paliwanag tungkol sa mga pangunahing bahagi ng aming sistema. Sa pamamagitan nito, mas madali mong magagamit ang mga feature at functionality na inaalok.

## Mga Seksyon

### Pangkalahatang-ideya

Ang seksyong ito ay nagbibigay ng buod ng mga pangunahing konsepto at kung paano ito nauugnay sa kabuuang sistema.

### Mga Detalye ng Pagpapatupad

Sa seksyong ito, tatalakayin natin ang mga teknikal na detalye na mahalaga para sa mga developer at iba pang teknikal na audience.

[!NOTE] Ang mga halimbawa sa ibaba ay para sa layuning pang-edukasyon lamang.

#### Halimbawa ng Code

Narito ang isang simpleng halimbawa ng code:

```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```

### Mga Madalas Itanong (FAQ)

#### Paano ko masisimulan?

Upang masimulan, sundin ang mga hakbang na nakasaad sa aming [Quick Start Guide](https://example.com/quick-start).

#### Ano ang mga kinakailangan?

Siguraduhing natutugunan ang mga sumusunod na kinakailangan bago gamitin ang sistema:

- Operating System: Windows, macOS, o Linux
- Minimum na RAM: 4GB
- Internet Connection

[!TIP] Para sa mas mabilis na pag-setup, gamitin ang aming auto-installer tool.

### Mga Karaniwang Isyu

#### Hindi gumagana ang sistema pagkatapos ng pag-install

Kung makakaranas ng problema pagkatapos ng pag-install, subukang i-restart ang iyong device at tiyaking nasusunod ang mga kinakailangan.

[!WARNING] Huwag baguhin ang mga configuration file maliban kung sigurado ka sa iyong ginagawa.

#### Nawawala ang mga file

Kung nawawala ang mga file, suriin ang iyong backup folder o gamitin ang aming recovery tool.

[!IMPORTANT] Regular na mag-backup ng iyong data upang maiwasan ang pagkawala ng mahalagang impormasyon.

## Konklusyon

Ang dokumentong ito ay naglalayong magbigay ng malinaw na gabay para sa paggamit ng aming sistema. Kung mayroon kang karagdagang tanong, huwag mag-atubiling makipag-ugnayan sa aming support team.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
